In [2]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
import yfinance as yf

In [3]:
yf.pdr_override()

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


Extração dos Dados para Visualização

In [4]:
tickers=['PG','BEI.DE']

sec_data= pd.DataFrame()

for t in tickers:
    sec_data[t]= wb.get_data_yahoo(t, start="2007-1-1")["Adj Close"]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
sec_data.tail()

,PG,BEI.DE
Date,,
2024-07-10,166.800003,138.399994
2024-07-11,165.539993,137.649994
2024-07-12,166.610001,138.350006
2024-07-15,164.580002,136.899994
2024-07-16,166.429993,135.850006


In [6]:
sec_returns= np.log(sec_data / sec_data.shift(1))

In [7]:
sec_returns

,PG,BEI.DE
Date,,
2007-01-03,NaN,NaN
2007-01-04,-0.007621,0.006544
2007-01-05,-0.008624,-0.020771
2007-01-08,0.002203,0.000202
2007-01-09,-0.002517,-0.022858
...,...,...
2024-07-10,0.006858,0.003982
2024-07-11,-0.007583,-0.005434
2024-07-12,0.006443,0.005073


Transformando os dados extraídos

.mean(): Média

.std: STANDARD DEVIATION
É a raíz quadrada da variância

Indice PG

In [8]:
sec_returns['PG'].mean()

0.00032843135644683555

In [9]:
# 250: Número de Operações anuais
sec_returns['PG'].mean() * 250

0.0821078391117089

In [10]:
sec_returns['PG'].std()

0.011632027431262562

In [11]:
sec_returns['PG'].std() * 250 ** 0.5

0.18391850244173688

Índice Beiersdorf

In [12]:
sec_returns['BEI.DE'].mean()

0.00025975330806307954

In [13]:
sec_returns['BEI.DE'].mean() * 250

0.06493832701576989

In [14]:
sec_returns['BEI.DE'].std()

0.01346999458687375

In [15]:
sec_returns['BEI.DE'].std() * 250 ** 0.5

0.2129793148232993

Resultado das duas para comparação

In [16]:
print(sec_returns['PG'].mean() * 250)
print(sec_returns['BEI.DE'].mean() * 250)

0.0821078391117089
0.06493832701576989


Resultado em 1 linha:

In [17]:
print(sec_returns[['PG','BEI.DE']].mean() * 250)

PG        0.082108
BEI.DE    0.064938
dtype: float64


Resultado em 1 linha

In [18]:
sec_returns[['PG', 'BEI.DE']].std() * 250 ** 0.5

PG        0.183919
BEI.DE    0.212979
dtype: float64

Calculando o Risco de um Portfólio

numpy.dot(): Nos gera o produto de duas arrays/ativos

Esquema/Array de Ações do mesmo peso:

In [19]:
weights= np.array([0.5, 0.5])

Variância do Portfólio:

In [20]:
pfolio_var= np.dot(weights.T, np.dot(sec_returns.cov() *250, weights))
pfolio_var

0.025077162914490532

Volatilidade do Portfólio:

In [23]:
pfolio_vol= (np.dot(weights.T, np.dot(sec_returns.cov()*250, weights))) ** 0.5
pfolio_vol

0.15835770557345966

In [24]:
print(str(round(pfolio_vol, 5)* 100) + '%')

15.836%


Cálculo do Risco Diversificável e Não Diversificável de um Portfólio:

In [25]:
weights= np.array([0.5, 0.5])

In [26]:
weights[0]

0.5

In [27]:
weights[1]

0.5

Risco Diversificado

risco diversificavel= variância_do_portfolio - variância_anual_ponderada

In [28]:
PG_var_a= sec_returns[['PG']].var() * 250
PG_var_a

PG    0.033826
dtype: float64

In [29]:
BEI_var_a= sec_returns[['BEI.DE']].var() * 250
BEI_var_a

BEI.DE    0.04536
dtype: float64

dr: diversified risk

In [30]:
dr= pfolio_vol - (weights[0] ** 2 * PG_var_a) - (weights[1] ** 2 * BEI_var_a)
dr

BEI.DE   NaN
PG       NaN
dtype: float64

In [31]:
float(PG_var_a)

C:\Users\Antonio\AppData\Local\Temp\ipykernel_15448\2706030445.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(PG_var_a)


0.033826015540411175

In [32]:
PG_var_a= sec_returns['PG'].var() * 250
PG_var_a

0.033826015540411175

In [33]:
BEI_var_a= sec_returns['BEI.DE'].var() * 250
BEI_var_a

0.045360188542602035

In [35]:
dr= pfolio_var - (weights[0] ** 2 * PG_var_a) - (weights[1] ** 2 * BEI_var_a)
dr

0.005280611893737229

In [36]:
print(str(round(dr*100, 3))+ '%')

0.528%


Risco Não-Sistemático

In [39]:
n_dr_1= pfolio_var - dr
n_dr_1

0.019796551020753304

In [37]:
n_dr_2= (weights[0] ** 2 * PG_var_a) + (weights[1] ** 2 * BEI_var_a)
n_dr_2

0.019796551020753304

In [40]:
n_dr_1 == n_dr_2

True